# Building CTMCs
A Continuous Time Markov Chain (CTMC) is similar to a DTMC. The differences are that time is *continuous*, not discrete, and that transitions have *rates* instead of probabilities. So to summarize, a CTMC has:
* states (includig an initial state)
* transitions with rates. The rate is an indication of the speed in which a transition occurs. To be precise, the probability that state $s_i$ goes to state $s_j$ in $t$ time steps is $1- e^{R(s_i,s_j)\cdot t}$, where $R(s_i,s_j)$ is the *rate* between $s_i$ and $s_j$.
* labels

As an example for a CTMC, we have a model of a star. It will first fuse hydrogen into helium until there is no hydrogen left, then it will fuse helium into carbon, etc. until there is only iron left and the star goes supernova.

In [3]:
import stormvogel.model
from stormvogel.show import show
from stormvogel.layout import Layout
# Create a new model with the name "Nuclear fusion"
ctmc = stormvogel.model.new_ctmc("Nuclear fusion")

# hydrogen fuses into helium
ctmc.get_state_by_id(0).set_transitions([(3, ctmc.new_state("helium"))])
# helium fuses into carbon
ctmc.get_state_by_id(1).set_transitions([(2, ctmc.new_state("carbon"))])
# carbon fuses into iron
ctmc.get_state_by_id(2).set_transitions([(7, ctmc.new_state("iron"))])
# supernova
ctmc.get_state_by_id(3).set_transitions([(12, ctmc.new_state("Supernova"))])

# we add the rates which are equal to whats in the transitions since the probabilities are all 1
rates = [3, 2, 7, 12, 0]
for i in range(5):
    ctmc.set_rate(ctmc.get_state_by_id(i), rates[i])

# we add self loops to all states with no outgoing transitions
ctmc.add_self_loops()
vis = show(ctmc, layout=Layout("layouts/star.json"))

Or using the pgc API

In [1]:
from stormvogel import pgc
from stormvogel.model import EmptyAction, ModelType
from stormvogel.show import show
from stormvogel.layout import Layout

init = pgc.State(x="")

def delta(s: pgc.State):
    if s == init:
        return [(3, pgc.State(x=["helium"]))]
    elif "helium" in s.x:
        return [(2, pgc.State(x=["carbon"]))]
    elif "carbon" in s.x:
        return [(7, pgc.State(x=["iron"]))]
    elif "iron" in s.x:
        return [(12, pgc.State(x=["Supernova"]))]
    else:
        return [(0, s)]
        
labels = lambda s: s.x

pgc_star = pgc.build_pgc(
    delta=delta,
    initial_state_pgc=init,
    labels=labels,
    modeltype=ModelType.CTMC
)

vis2 = show(pgc_star, layout=Layout("layouts/star.json"))